<a href="https://www.kaggle.com/code/kajuyerim/lora-guide-on-llama3-1-8b-instruct?scriptVersionId=191363629" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# LoRA Guide About Sentimental Analysis on Financial Domain Using Llama3.1 8b-instruct
---

## What is LoRA?
* #### Low Rank Adaptation: Freezing the pretrained model weights and injecting trainable rank decomposition matrices into each layer of the Transformer architecture.
* #### Main purpose is to lower the dimensions of the matrix by 
* #### Instead of updating all weights of a model, we only update the injected low rank matrices.

## Why Use LoRA?
* #### Greatly reducing the number of trainable parameters (up to 10000 times).
* #### Reducing the GPU memory requirement (up to 3 times).

---
## ! This guide is based on the LoRA paper (LoRA: Low-Rank Adaptation for Neural Networks)(https://arxiv.org/abs/2106.09685#). 
## ! Any mistakes between this guide and the paper are due to my interpretation on the material.
---


In [2]:
!pip install -U transformers datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 72.4 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.3
    Uninstalling transformers-4.42.3:
      Successfully uninstalled transformers-4.42.3


We need to prepare the dataset and load it so that we can use it to train our model with LoRA.

In [3]:
# Re-import necessary libraries since the code execution state was reset.
from datasets import load_dataset

# Load and shuffle the dataset with a specified configuration
dataset = load_dataset("takala/financial_phrasebank", "sentences_allagree", split='train')
shuffled_dataset = dataset.shuffle(seed=50)

# Map numerical labels to string labels
label_mapping = {0: "negative", 1: "neutral", 2: "positive"}

# Format examples into "sentence" - "sentiment" format
formatted_examples = [
    f'"{example["sentence"]}" - "{label_mapping[example["label"]]}"' for example in shuffled_dataset
]

# Print the first 5 formatted examples
for example in formatted_examples[:10]:
    print(example)


The repository for takala/financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/takala/financial_phrasebank.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

"Deliveries of Nokia 1112 , Nokia 2310 and Nokia 2610 are expected to start in the second quarter of 2006 ." - "neutral"
"Any investment or investment activity to which this communication relates is only available to relevant persons and will be engaged in only with relevant persons ." - "neutral"
"The operations to be sold include manufacturing units in Finland , France , Poland and Turkey , as well as sales units in Germany and Lithuania ." - "neutral"
"Ruukki 's delivery volumes and selling prices showed favourable development and the company 's comparable net sales grew by 50 % year-on-year to EUR647m , CEO Sakari Tamminen said ." - "positive"
"The company serves customers in various industries , including process and resources , industrial machinery , architecture , building , construction , electrical , transportation , electronics , chemical , petrochemical , energy , and information technology , as well as catering and households ." - "neutral"
"In the reporting period , the co

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

login(token = hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from transformers import AutoTokenizer,AutoModelForCausalLM,pipeline
import torch

base_model = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(base_model)

model = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ValueError: `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}